<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


## Introduction


As we discussed in the videos, despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

So, in this lab, you will learn how to use the Keras library to build a regression model.


<h2>Regression Models with Keras</h2>

<h3>Objective for this Notebook<h3>    
<h5> 1. How to use the Keras library to build a regression model.</h5>
<h5> 2. Download and Clean dataset </h5>
<h5> 3. Build a Neural Network </h5>
<h5> 4. Train and Test the Network. </h5>     


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [14]:
n_cols = predictors_norm.shape[1] # number of predictors

n_rows = predictors_norm.shape[0]

n_rows, n_cols

(1030, 8)

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [15]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [16]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [17]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [18]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [19]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)



Train on 721 samples, validate on 309 samples
Epoch 1/100


2021-12-03 05:18:22.414509: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2021-12-03 05:18:22.420185: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394300000 Hz
2021-12-03 05:18:22.421111: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564f4a3d9a20 executing computations on platform Host. Devices:
2021-12-03 05:18:22.421153: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-03 05:18:22.514276: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1667.4191 - val_loss: 1153.6797
Epoch 2/100
 - 0s - loss: 1541.7366 - val_loss: 1035.4510
Epoch 3/100
 - 0s - loss: 1342.9077 - val_loss: 853.8355
Epoch 4/100
 - 0s - loss: 1037.5979 - val_loss: 617.8041
Epoch 5/100
 - 0s - loss: 672.3409 - val_loss: 385.9166
Epoch 6/100
 - 0s - loss: 382.1068 - val_loss: 239.7421
Epoch 7/100
 - 0s - loss: 256.1238 - val_loss: 185.3997
Epoch 8/100
 - 0s - loss: 224.9511 - val_loss: 169.6135
Epoch 9/100
 - 0s - loss: 211.4114 - val_loss: 163.3060
Epoch 10/100
 - 0s - loss: 199.4234 - val_loss: 162.4895
Epoch 11/100
 - 0s - loss: 191.3458 - val_loss: 160.0450
Epoch 12/100
 - 0s - loss: 184.3526 - val_loss: 158.1600
Epoch 13/100
 - 0s - loss: 178.7593 - val_loss: 159.0702
Epoch 14/100
 - 0s - loss: 174.2952 - val_loss: 158.5259
Epoch 15/100
 - 0s - loss: 169.5534 - val_loss: 158.8046
Epoch 16/100
 - 0s - loss: 165.1544 - val_loss: 157.2591
Epoch 17/100
 - 0s - loss: 161.9716 - val_loss: 158.2358
Epoch 18/100
 - 0s - loss: 158.5749 - val_loss

<strong>You can refer to this [link](https://keras.io/models/sequential?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) to learn about other functions that you can use for prediction or evaluation.</strong>


Feel free to vary the following and note what impact each change has on the model's performance:

1.  Increase or decreate number of neurons in hidden layers
2.  Add more hidden layers
3.  Increase number of epochs


In [21]:
# define regression model
def regression_model1():
    # create model
    model1 = Sequential()
    model1.add(Dense(200, activation='relu', input_shape=(n_cols,)))
    model1.add(Dense(200, activation='relu'))
    model1.add(Dense(1))
    
    # compile model
    model1.compile(optimizer='adam', loss='mean_squared_error')
    return model1

# build the model
model1 = regression_model1()

# fit the model
model1.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 1500.2310 - val_loss: 856.6704
Epoch 2/100
 - 0s - loss: 842.8643 - val_loss: 332.9033
Epoch 3/100
 - 0s - loss: 291.9150 - val_loss: 179.0050
Epoch 4/100
 - 0s - loss: 227.1627 - val_loss: 172.2916
Epoch 5/100
 - 0s - loss: 200.5386 - val_loss: 165.3853
Epoch 6/100
 - 0s - loss: 185.7659 - val_loss: 165.5551
Epoch 7/100
 - 0s - loss: 176.2334 - val_loss: 163.6874
Epoch 8/100
 - 0s - loss: 169.0094 - val_loss: 166.2231
Epoch 9/100
 - 0s - loss: 162.9288 - val_loss: 166.2833
Epoch 10/100
 - 0s - loss: 157.6614 - val_loss: 165.2531
Epoch 11/100
 - 0s - loss: 153.5864 - val_loss: 166.9894
Epoch 12/100
 - 0s - loss: 150.5089 - val_loss: 168.6654
Epoch 13/100
 - 0s - loss: 147.3272 - val_loss: 171.0276
Epoch 14/100
 - 0s - loss: 145.1458 - val_loss: 170.6200
Epoch 15/100
 - 0s - loss: 142.6257 - val_loss: 173.4189
Epoch 16/100
 - 0s - loss: 140.8135 - val_loss: 169.4532
Epoch 17/100
 - 0s - loss: 137.8921 - val_loss: 17

In [24]:
# define regression model
def regression_model2():
    # create model
    model2 = Sequential()
    model2.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model2.add(Dense(50, activation='relu'))
    model2.add(Dense(60, activation='relu'))
    model2.add(Dense(40, activation='relu'))
    model2.add(Dense(50, activation='relu'))
    model2.add(Dense(60, activation='relu'))
    model2.add(Dense(40, activation='relu'))
    model2.add(Dense(50, activation='relu'))
    model2.add(Dense(1))
    
    # compile model
    model2.compile(optimizer='adam', loss='mean_squared_error')
    return model2

# build the model
model2 = regression_model2()

# fit the model
model2.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 1s - loss: 1672.2840 - val_loss: 1108.6073
Epoch 2/100
 - 0s - loss: 908.3012 - val_loss: 360.9406
Epoch 3/100
 - 0s - loss: 279.6000 - val_loss: 246.0730
Epoch 4/100
 - 0s - loss: 233.7917 - val_loss: 250.7529
Epoch 5/100
 - 0s - loss: 209.9175 - val_loss: 253.9343
Epoch 6/100
 - 0s - loss: 177.6313 - val_loss: 213.3819
Epoch 7/100
 - 0s - loss: 155.8291 - val_loss: 206.0130
Epoch 8/100
 - 0s - loss: 133.7449 - val_loss: 184.4198
Epoch 9/100
 - 0s - loss: 114.1684 - val_loss: 195.0676
Epoch 10/100
 - 0s - loss: 96.3100 - val_loss: 147.1262
Epoch 11/100
 - 0s - loss: 84.2986 - val_loss: 165.6697
Epoch 12/100
 - 0s - loss: 70.2230 - val_loss: 145.0185
Epoch 13/100
 - 0s - loss: 64.5466 - val_loss: 160.1113
Epoch 14/100
 - 0s - loss: 60.1564 - val_loss: 140.9531
Epoch 15/100
 - 0s - loss: 53.3017 - val_loss: 140.7181
Epoch 16/100
 - 0s - loss: 49.3186 - val_loss: 156.9717
Epoch 17/100
 - 0s - loss: 45.9259 - val_loss: 142.0747


In [25]:
# define regression model
def regression_model3():
    # create model
    model3 = Sequential()
    model3.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model3.add(Dense(50, activation='relu'))
    model3.add(Dense(1))
    
    # compile model
    model3.compile(optimizer='adam', loss='mean_squared_error')
    return model1

# build the model
model3 = regression_model3()

# fit the model
model3.fit(predictors_norm, target, validation_split=0.3, epochs=500, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/500
 - 0s - loss: 19.4043 - val_loss: 149.3994
Epoch 2/500
 - 0s - loss: 19.6551 - val_loss: 150.1243
Epoch 3/500
 - 0s - loss: 18.8552 - val_loss: 144.3688
Epoch 4/500
 - 0s - loss: 20.4744 - val_loss: 156.1171
Epoch 5/500
 - 0s - loss: 19.8371 - val_loss: 152.6368
Epoch 6/500
 - 0s - loss: 18.4998 - val_loss: 136.0929
Epoch 7/500
 - 0s - loss: 19.2406 - val_loss: 151.0831
Epoch 8/500
 - 0s - loss: 19.0157 - val_loss: 145.0285
Epoch 9/500
 - 0s - loss: 17.7490 - val_loss: 164.9512
Epoch 10/500
 - 0s - loss: 18.2152 - val_loss: 153.4209
Epoch 11/500
 - 0s - loss: 18.0603 - val_loss: 156.2559
Epoch 12/500
 - 0s - loss: 17.7549 - val_loss: 154.0085
Epoch 13/500
 - 0s - loss: 17.6560 - val_loss: 183.7290
Epoch 14/500
 - 0s - loss: 20.2660 - val_loss: 155.7071
Epoch 15/500
 - 0s - loss: 18.4409 - val_loss: 158.3092
Epoch 16/500
 - 0s - loss: 17.5888 - val_loss: 151.7970
Epoch 17/500
 - 0s - loss: 17.0988 - val_loss: 155.9707
Epoch 18/50

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                          |
| ----------------- | ------- | ---------- | ----------------------------------------------------------- |
| 2020-09-21        | 2.0     | Srishti    | Migrated Lab to Markdown and added to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>


This notebook is part of a course on **Coursera** called _Introduction to Deep Learning & Neural Networks with Keras_. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).


<hr>

Copyright © 2019 [IBM Developer Skills Network](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
